##### Capstone Project1: Identifying counties that are the most fire-prone and predicting the cause of a fire wildfire. 

DataSet: 1.88 Million US Wildfires (Kaggle1)

This data publication contains a spatial database of wildfires that occurred in the United States from 1992 to 2015. It is the third update of a publication originally generated to support the national Fire Program Analysis (FPA) system. The wildfire records were acquired from the reporting systems of federal, state, and local fire organizations. The following core data elements were required for records to be included in this data publication: discovery date, final fire size, and a point location at least as precise as Public Land Survey System (PLSS) section (1-square mile grid). The data were transformed to conform, when possible, to the data standards of the National Wildfire Coordinating Group (NWCG). Basic error-checking was performed, and redundant records were identified and removed, to the degree possible. The resulting product, referred to as the Fire Program Analysis fire-occurrence database (FPA FOD), includes 1.88 million geo-referenced wildfire records, representing a total of 140 million acres burned during the 24 years.


Approach: In this notebook, I will do the below steps:

        Fetch Data and create a Pandas Dataframe for further analysis
        Data Wrangling to clean and transform the data


##Step1: Fetch Data from Database

As the dataset was in SQL file format, I did some cleaning activity in SQLite. There were many columns in SQL table "fires” which were related to Source of data and agency information which collected or prepared reports on Data. I omitted these columns and imported only relevant data to the "US_Wildfire_data.csv" file. Also while importing I filled in all the null/blank values with "Nan" value.


select FOD_ID,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID
,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR ,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,
FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME from Fires

In [2]:
import pandas as pd
import numpy as np
US_fires = pd.read_csv('US_Wildfire_data.csv',low_memory=False)

In [3]:
US_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 29 columns):
FOD_ID                     1048575 non-null int64
LOCAL_FIRE_REPORT_ID       301217 non-null object
LOCAL_INCIDENT_ID          369317 non-null object
FIRE_CODE                  223449 non-null object
FIRE_NAME                  412256 non-null object
ICS_209_INCIDENT_NUMBER    12544 non-null object
ICS_209_NAME               12544 non-null object
MTBS_ID                    6989 non-null object
MTBS_FIRE_NAME             6989 non-null object
COMPLEX_NAME               3173 non-null object
FIRE_YEAR                  1048575 non-null int64
DISCOVERY_DATE             1048575 non-null float64
DISCOVERY_DOY              1048575 non-null int64
DISCOVERY_TIME             434827 non-null float64
STAT_CAUSE_CODE            1048575 non-null int64
STAT_CAUSE_DESCR           1048575 non-null object
CONT_DATE                  462280 non-null float64
CONT_DOY                   462280 non-nu

##Step2: 

By looking at the info of the dataframe I observed many columns with more than 50% of its total data with null values. As those columns were not helping the data set I omitted those columns.


In [4]:
fires_data = US_fires[['FOD_ID','DISCOVERY_DATE','FIRE_YEAR','DISCOVERY_DOY','STAT_CAUSE_CODE','STAT_CAUSE_DESCR','FIRE_SIZE','FIRE_SIZE_CLASS','LATITUDE','LONGITUDE','OWNER_CODE','OWNER_DESCR','STATE','COUNTY','FIPS_CODE','FIPS_NAME']]

In [5]:
fires_data.head(10)

,FOD_ID,DISCOVERY_DATE,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,2453403.5,2005,33,9,Miscellaneous,0.10,A,40.036944,-121.005833,5,USFS,CA,63,63.0,Plumas
1,2,2453137.5,2004,133,1,Lightning,0.25,A,38.933056,-120.404444,5,USFS,CA,61,61.0,Placer
2,3,2453156.5,2004,152,5,Debris Burning,0.10,A,38.984167,-120.735556,13,STATE OR PRIVATE,CA,17,17.0,El Dorado
3,4,2453184.5,2004,180,1,Lightning,0.10,A,38.559167,-119.913333,5,USFS,CA,3,3.0,Alpine
4,5,2453184.5,2004,180,1,Lightning,0.10,A,38.559167,-119.933056,5,USFS,CA,3,3.0,Alpine
5,6,2453186.5,2004,182,1,Lightning,0.10,A,38.635278,-120.103611,5,USFS,CA,5,5.0,Amador
6,7,2453187.5,2004,183,1,Lightning,0.10,A,38.688333,-120.153333,5,USFS,CA,17,17.0,El Dorado
7,8,2453437.5,2005,67,5,Debris Burning,0.80,B,40.968056,-122.433889,13,STATE OR PRIVATE,CA,NaN,NaN,NaN
8,9,2453444.5,2005,74,5,Debris Burning,1.00,B,41.233611,-122.283333,13,STATE OR PRIVATE,CA,NaN,NaN,NaN
9,10,2453187.5,2004,183,1,Lightning,0.10,A,38.548333,-120.149167,5,USFS,CA,5,5.0,Amador


Step3:

 As County name is the column which I am interested in, I checked if there are any Nan values in that column. I found almost 50% of data missing in the column. I tried getting the county name with Latitude and Longitude available using urllib and reverse_geocoder. But for half a million data rows either it was entering a time loop or giving different data than expected. Hence I decided to remove Nan rows with respect to FIPS_NAME(county name).


In [6]:
print(fires_data['FIPS_NAME'].value_counts(dropna=False))     

NaN             489154
Coconino          6190
Lincoln           5646
Polk              5512
Jackson           5415
                 ...  
Denver               1
Schuyler             1
Hampton City         1
Wilkin               1
Edmunds              1
Name: FIPS_NAME, Length: 1427, dtype: int64


In [7]:
cond = fires_data['FIPS_NAME'].isnull()
fires_data.drop(fires_data[cond].index,inplace = True)

C:\Users\Lasya\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
fires_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559421 entries, 0 to 997173
Data columns (total 16 columns):
FOD_ID              559421 non-null int64
DISCOVERY_DATE      559421 non-null float64
FIRE_YEAR           559421 non-null int64
DISCOVERY_DOY       559421 non-null int64
STAT_CAUSE_CODE     559421 non-null int64
STAT_CAUSE_DESCR    559421 non-null object
FIRE_SIZE           559421 non-null float64
FIRE_SIZE_CLASS     559421 non-null object
LATITUDE            559421 non-null float64
LONGITUDE           559421 non-null float64
OWNER_CODE          559421 non-null int64
OWNER_DESCR         559421 non-null object
STATE               559421 non-null object
COUNTY              559421 non-null object
FIPS_CODE           559421 non-null float64
FIPS_NAME           559421 non-null object
dtypes: float64(5), int64(5), object(6)
memory usage: 72.6+ MB


In [9]:
fires_data.shape

(559421, 16)

In [10]:
fires_data.head()

,FOD_ID,DISCOVERY_DATE,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,2453403.5,2005,33,9,Miscellaneous,0.10,A,40.036944,-121.005833,5,USFS,CA,63,63.0,Plumas
1,2,2453137.5,2004,133,1,Lightning,0.25,A,38.933056,-120.404444,5,USFS,CA,61,61.0,Placer
2,3,2453156.5,2004,152,5,Debris Burning,0.10,A,38.984167,-120.735556,13,STATE OR PRIVATE,CA,17,17.0,El Dorado
3,4,2453184.5,2004,180,1,Lightning,0.10,A,38.559167,-119.913333,5,USFS,CA,3,3.0,Alpine
4,5,2453184.5,2004,180,1,Lightning,0.10,A,38.559167,-119.933056,5,USFS,CA,3,3.0,Alpine


##Step4: 

column "DISCOVERY_DATE" was in data type float64 and not in Datetime format as expected. Hence I converted it to a readable date format from julian date.Dropped DISCOVERY_DATE. Remaned "date" column and FOD_ID column names to more appropriate names.


In [11]:
from datetime import datetime
epoch = pd.to_datetime(0, unit='s').to_julian_date()
fires_data['date'] = pd.to_datetime(fires_data['DISCOVERY_DATE']- epoch, unit='D')

C:\Users\Lasya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
fires_data[['date','DISCOVERY_DATE']]

,date,DISCOVERY_DATE
0,2005-02-02,2453403.5
1,2004-05-12,2453137.5
2,2004-05-31,2453156.5
3,2004-06-28,2453184.5
4,2004-06-28,2453184.5
...,...,...
997169,2001-06-04,2452064.5
997170,2000-07-19,2451744.5
997171,2002-07-03,2452458.5
997172,2000-06-17,2451712.5


In [13]:
fires_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559421 entries, 0 to 997173
Data columns (total 17 columns):
FOD_ID              559421 non-null int64
DISCOVERY_DATE      559421 non-null float64
FIRE_YEAR           559421 non-null int64
DISCOVERY_DOY       559421 non-null int64
STAT_CAUSE_CODE     559421 non-null int64
STAT_CAUSE_DESCR    559421 non-null object
FIRE_SIZE           559421 non-null float64
FIRE_SIZE_CLASS     559421 non-null object
LATITUDE            559421 non-null float64
LONGITUDE           559421 non-null float64
OWNER_CODE          559421 non-null int64
OWNER_DESCR         559421 non-null object
STATE               559421 non-null object
COUNTY              559421 non-null object
FIPS_CODE           559421 non-null float64
FIPS_NAME           559421 non-null object
date                559421 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(5), object(6)
memory usage: 76.8+ MB


Dropped DISCOVERY_DATE. Remaned "date" column  and FOD_ID column names to more appropiate names.           

In [14]:
fires_data = fires_data.drop(labels=['DISCOVERY_DATE'],axis=1)
fires_data = fires_data.rename(columns={'date':'DISCOVERY_DATE','FOD_ID':'GLOBAL_UNIQUE_ID'})
fires_data.head()

,GLOBAL_UNIQUE_ID,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,DISCOVERY_DATE
0,1,2005,33,9,Miscellaneous,0.10,A,40.036944,-121.005833,5,USFS,CA,63,63.0,Plumas,2005-02-02
1,2,2004,133,1,Lightning,0.25,A,38.933056,-120.404444,5,USFS,CA,61,61.0,Placer,2004-05-12
2,3,2004,152,5,Debris Burning,0.10,A,38.984167,-120.735556,13,STATE OR PRIVATE,CA,17,17.0,El Dorado,2004-05-31
3,4,2004,180,1,Lightning,0.10,A,38.559167,-119.913333,5,USFS,CA,3,3.0,Alpine,2004-06-28
4,5,2004,180,1,Lightning,0.10,A,38.559167,-119.933056,5,USFS,CA,3,3.0,Alpine,2004-06-28


Step5: Rearraged columns in dataframe

In [15]:
column_names = ["GLOBAL_UNIQUE_ID", "DISCOVERY_DATE",'FIRE_YEAR', "STAT_CAUSE_CODE",'STAT_CAUSE_DESCR','FIRE_SIZE','FIRE_SIZE_CLASS','STATE','COUNTY','FIPS_CODE','FIPS_NAME','LATITUDE','LONGITUDE','DISCOVERY_DOY','OWNER_CODE','OWNER_DESCR']
fires_data = fires_data.reindex(columns=column_names)

In [16]:
fires_data.head()

,GLOBAL_UNIQUE_ID,DISCOVERY_DATE,FIRE_YEAR,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,STATE,COUNTY,FIPS_CODE,FIPS_NAME,LATITUDE,LONGITUDE,DISCOVERY_DOY,OWNER_CODE,OWNER_DESCR
0,1,2005-02-02,2005,9,Miscellaneous,0.10,A,CA,63,63.0,Plumas,40.036944,-121.005833,33,5,USFS
1,2,2004-05-12,2004,1,Lightning,0.25,A,CA,61,61.0,Placer,38.933056,-120.404444,133,5,USFS
2,3,2004-05-31,2004,5,Debris Burning,0.10,A,CA,17,17.0,El Dorado,38.984167,-120.735556,152,13,STATE OR PRIVATE
3,4,2004-06-28,2004,1,Lightning,0.10,A,CA,3,3.0,Alpine,38.559167,-119.913333,180,5,USFS
4,5,2004-06-28,2004,1,Lightning,0.10,A,CA,3,3.0,Alpine,38.559167,-119.933056,180,5,USFS


Step6: Checked if there are any outliers with respect to FIRE_YEAR column

In [17]:
fires_data['FIRE_YEAR'].describe()

count    559421.000000
mean       2002.127591
std           5.256793
min        1992.000000
25%        1998.000000
50%        2004.000000
75%        2006.000000
max        2009.000000
Name: FIRE_YEAR, dtype: float64

there were no outliers which needs to be discarded

In [18]:
fires_data.to_csv('fires_clean_data.csv')